## PDF Query Using Langchain

In [1]:
# !pip install langchain
# !pip install openai
# !pip install PyPDF2
# !pip install faiss-cpu
# !pip install tiktoken

In [2]:
from secret_key import openapi_key
import os 
os.environ['OPENAI_API_KEY'] = openapi_key


In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [4]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('MachineLearning-Lecture01.pdf')

In [5]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [6]:
print(raw_text)

MachineLearning-Lecture01  
Instructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is ju st spend a little time going over the logistics 
of the class, and then we'll start to  talk a bit about machine learning.  
By way of introduction, my name's  Andrew Ng and I'll be instru ctor for this class. And so 
I personally work in machine learning, and I' ve worked on it for about 15 years now, and 
I actually think that machine learning is th e most exciting field of all the computer 
sciences. So I'm actually always excited about  teaching this class. Sometimes I actually 
think that machine learning is not only the most exciting thin g in computer science, but 
the most exciting thing in all of human e ndeavor, so maybe a little bias there.  
I also want to introduce the TAs, who are all graduate students doing research in or 
related to the machine learni ng and all aspects of machin e learning. Paul Baumstarck 
works in mach

In [7]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [8]:
len(texts)

104

In [9]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
document_search = FAISS.from_texts(texts, embeddings)

In [11]:
document_search


In [12]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [13]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [14]:
query = "Vision for MI"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' In this context, it is not clear what "MI" stands for. It could potentially stand for "machine intelligence" or "medical imaging," but without more information, it is impossible to provide an accurate answer. Additionally, the given context does not mention anything about a specific vision for MI.'

In [15]:
query = "major MI applications"
docs = document_search.similarity_search(query)
print(chain.run(input_documents=docs, question=query))

 Some of the most common applications for machine learning include improving state-of-the-art machine learning, developing the theory of machine learning, and proving theorems about machine learning. It is also used in various industries such as computer vision, biology, robotics, language, and economics. Additionally, it has made significant advancements in database mining, particularly in the healthcare industry.


In [16]:
query = "instructors"
docs = document_search.similarity_search(query)
print(chain.run(input_documents=docs, question=query))

 The instructors for this class are Andrew Ng and a team of TAs, including Paul Baumstarck, Catie Chang, Tom Do, and Zico Kolter.


In [18]:
query = "authors"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' It is not possible to determine the authors of the text without more information.'

In [19]:
from langchain.document_loaders import OnlinePDFLoader